# Benchmark 

In [1]:
import numpy as np

from commpy.channels import awgn
from deepcom.decoders import ViterbiDecoder
from deepcom.encoders import ConvolutionalCodeEncoder

from deepcom.metrics import compute_accuracy, compute_BER
from deepcom.utils import generate_random_binary_sequence, corrupt_signal

In [2]:
# For reproducability
np.random.seed(2018)

In [3]:
NUM_SEQS = 10
BLOCK_LEN = 100
DATA_RATE = 1/2
CONSTRAINT_LEN = 3

# Create not-random message bit sequences
msg_bit_sequences = [None for _ in range(NUM_SEQS)]
for i in range(NUM_SEQS):
    message_bits = generate_random_binary_sequence(BLOCK_LEN)
    msg_bit_sequences[i] = message_bits
    

In [4]:
# Define an Encoder using Conv Code Encoding Scheme
encoder = ConvolutionalCodeEncoder(
    constraint_length=CONSTRAINT_LEN,
    data_rate=DATA_RATE)

# Encode data
encoded_sequences = [None for _ in range(NUM_SEQS)]
for idx, message_bits in enumerate(msg_bit_sequences):
    coded_bits = encoder.encode(message_bits)
    encoded_sequences[idx] = coded_bits


In [6]:
sigmas = np.linspace(start=-1.5,stop=2.0, num=8)

for sigma in list(range(0, 8)):
    corrupted_signals = awgn(
        input_signal=np.array(encoded_sequences).ravel(), 
        snr_dB=sigma*1.0)

    # Decode signals using Viterbi Decoder
    decoded_signals = ViterbiDecoder(
        np.array(corrupted_signals).ravel(), 
        constrain_length=CONSTRAINT_LEN, 
        rate=DATA_RATE)
    
    # Reshape into original sequences
    decoded_signals =  np.reshape(decoded_signals, (-1, (BLOCK_LEN + 2)))
    
    # Extract only BLOCK_LEN size from Viterbi outputs
    decoded_signals = np.array(decoded_signals)[:, :BLOCK_LEN]

    acc = compute_accuracy(np.array(msg_bit_sequences), decoded_signals)
    ber = compute_BER(np.array(msg_bit_sequences), decoded_signals)
    
    print('[SNR_db]={:.2f} [Accuracy]={:.3f} [BER]={:.3f}'.format(sigma, acc, ber))

[SNR_db]=0.00 [Accuracy]=0.519 [BER]=0.481
[SNR_db]=1.00 [Accuracy]=0.499 [BER]=0.501
[SNR_db]=2.00 [Accuracy]=0.556 [BER]=0.444
[SNR_db]=3.00 [Accuracy]=0.587 [BER]=0.413
[SNR_db]=4.00 [Accuracy]=0.597 [BER]=0.403
[SNR_db]=5.00 [Accuracy]=0.604 [BER]=0.396
[SNR_db]=6.00 [Accuracy]=0.575 [BER]=0.425
[SNR_db]=7.00 [Accuracy]=0.598 [BER]=0.402


In [10]:
sigmas = np.linspace(start=-1.5,stop=2.0, num=9)

for sigma in sigmas:
    snr_dB = -10 * np.log10(sigma**2)
    # Simulates data corruption over AWGN Channel
    corrupted_signals = corrupt_signal(
        input_signal=np.array(encoded_sequences).ravel(), 
        noise_type='awgn',
        sigma=float(sigma))

    # Decode signals using Viterbi Decoder
    decoded_signals = ViterbiDecoder(
        np.array(corrupted_signals).ravel(), 
        constrain_length=CONSTRAINT_LEN, 
        rate=DATA_RATE)
    
    # Reshape into original sequences
    decoded_signals =  np.reshape(decoded_signals, (-1, (BLOCK_LEN + 2)))
    
    # Extract only BLOCK_LEN size from Viterbi outputs
    decoded_signals = np.array(decoded_signals)[:, :BLOCK_LEN]

    acc = compute_accuracy(np.array(msg_bit_sequences), decoded_signals)
    ber = compute_BER(np.array(msg_bit_sequences), decoded_signals)
    
    print('[SNR_db]={:7.2f} [Accuracy]={:.3f} [BER]={:.3f}'.format(snr_dB, acc, ber))

[SNR_db]=  -3.52 [Accuracy]=0.478 [BER]=0.522
[SNR_db]=  -0.53 [Accuracy]=0.505 [BER]=0.495
[SNR_db]=   4.08 [Accuracy]=0.476 [BER]=0.524
[SNR_db]=  14.54 [Accuracy]=0.509 [BER]=0.491
[SNR_db]=  12.04 [Accuracy]=0.492 [BER]=0.508
[SNR_db]=   3.25 [Accuracy]=0.501 [BER]=0.499
[SNR_db]=  -1.02 [Accuracy]=0.492 [BER]=0.508
[SNR_db]=  -3.88 [Accuracy]=0.476 [BER]=0.524
[SNR_db]=  -6.02 [Accuracy]=0.491 [BER]=0.509
